# Whispers of the Heart - A daily reflection journal

Journaling allows individuals to reflect on their thoughts, feelings, and experiences, promoting self-awareness and personal growth. As journals creates a safe space for a person to freely express their thoughts and feelings, therapists often use journals as a therapeutic tool to gain deeper insights into their clients' thoughts, feelings, and experiences.

## Library Imports

In [2]:
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Importing the necessary libraries
import pandas as pd
from datetime import date
import os

from IPython.display import display, Markdown
import time
# from matplotlib import pyplot as plt

from openai import AzureOpenAI

import requests
import base64

In [4]:
# The API key import for the OpenAI model
# Please use the submitted API key for running the model.
#
# If you are running this on Google Colab, use the Secrets panel
# (the key icon on the left panel) and add the API key, with the
# name GOOGLE_API_KEY and the key as the value. Switch on the
# Notebook Access button.

OPENAI_ENDPOINT=os.environ("OPENAI_ENDPOINT")
OPENAI_API_KEY=os.environ("OPENAI_API_KEY")

## Initialising the variables

In [5]:
# Initialising the names of the files

data_folder = "data"
journal_file = "journal.txt"
analysis_file = "analysis.txt"

if "data" not in os.listdir():
  os.mkdir("data")

In [6]:
# Helper function to return the day of the week given the date

def day_of_the_week(date):
  weekday = date.weekday()
  days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
  return days[weekday]


# Initialising today's date

today = date.today()
date_display = str(day_of_the_week(today)) + " - " + str(today.strftime("%B %d, %Y"))

In [7]:
# Defining the prompt for the journal analysis

journal_context = '''

You are part of a journalling app, where the person will write about his everyday experience and share his thoughts and feelings.
But you are more than any ordinary journal. You will track the emotions and thoughts of the person and make observations out of it.

Finally you will summarise the person's mental state for the day.

The response should not make any conclusive decision about what the person should do, as that is the therapist's job.
The response should be an observation of the person's mental well-being and making an analysis of it, to help the therapist come up with the decisions.

You can use the previous day's journal entry to have a progression in emotions, but the analysis and the summary has to be only of today's journal entry.

The structure of the response should be:

**Emotions**: <A list of atleast 3 emotions they are going through.>
**Possible thought patterns**: <A collection of 3-4 prominent thought patterns the person is having with a brief explanation.>

**Mental well-being scores:**
<A list of 10-15 mental states of the person, with two scores - confidence (how sure you are about the state of the person) and intensity (the strength of the emotion in the person).
Make both the scores out of 10. Include all major mental health states.
Make it look like a table.
Sort the states in the decreasing order of the confidence scores>

**Summary of the day:** <The summary should be informative to the therapist. Highlight the progresson of the person's mental state from the previous days to the current day. Keep the summary in 3-4 sentences/>

**Journal Excerpts:** <Top 3 excerpts of the journal that helped you make these analysis. Take only a small snippet (or a few short phrases) of the journal entry for each excerpt. Break down your reasoning step-by-step. Give clarifications or explanations for the analysis. DO NOT display the personal identification information of the person or anyone that they speak about. Keep the privacy intact. You can replace their information with placeholders.>

Previous days' journal entry:
{journal_yesterday}
Today's journal entry:
{journal_today}
'''

In [8]:
journal_prompt_template = '''

Previous days' journal entry:
{journal_yesterday}

Today's journal entry:
{journal_today}
'''

In [9]:
# Defining hte prompt for the analysis of drawing

drawing_context = '''
You are a part of a journalling app, where the person will draw an image to convey his thoughts, feelings and mood.
But you are more than any ordinary journal. You will analyse and understand the emotions and thoughts of the person through the image.

You will also score the person's mental well-being on a few mental health 'symptoms' on a scale of 1 to 10 on how likely the person is in this state.
Finally you will summarise the person's mental state for the day.

The response should NOT make any conclusive decision about what the person should do, as that is the therapist's job.
The response should be an observation of the person's mental well-being and making an analysis of it, to help the therapist come up with the decisions.
The response should analyse the image and what the image is conveying and how the image is reflection of the person.

The structure of the response should be:

1. **Emotions**: <A list of atleast 3 emotions the image depicts.>
2. **Possible thought patterns**: <A collection of 3-4 prominent thought patterns the image is having with a brief explanation.>
3. **Mental well-being scores:**
<A list of 10-15 mental states as indicated by the image, with two scores - confidence (how sure you are about the state in the image) and intensity (the strength of the emotion in the image).
Make both the scores out of 10. Include all major mental health states.
Make it look like a table.
Sort the states in the decreasing order of the confidence scores>
4. **Summary of the day:** <The summary should be informative to the therapist. Keep the summary in 3-4 sentences/>
5. **Explanation:** <Explain your reasoning with detailed descriptions of the image. Break down your reasoning step-by-step. >
'''

In [10]:
# Defining hte prompt for the analysis of the video
video_context = '''
You are a part of a journalling app, where the person will send you a video to convey his thoughts, feelings and mood.
But you are more than any ordinary journal. You will analyse and understand the emotions and thoughts of the person who is listening to the video.

You will also score the person's mental well-being on a few mental health 'symptoms' on a scale of 1 to 10 on how likely the person is in this state.
Finally you will summarise the person's mental state for the day.

The response should NOT make any conclusive decision about what the person should do, as that is the therapist's job.
The response should be an observation of the person's mental well-being and making an analysis of it, to help the therapist come up with the decisions.

The structure of the response should be:

1. **Emotions**: <A list of atleast 3 emotions the person is in.>
2. **Possible thought patterns**: <A collection of 3-4 prominent thought patterns the person is having with a brief explanation.>
3. **Mental well-being scores:**
<A list of 10-15 mental states indicated by the person, with two scores - confidence (how sure you are about the state) and intensity (the strength of the state).
Make both the scores out of 10. Include all major mental health states.
Make it look like a table.
Sort the states in the decreasing order of the confidence scores>
4. **Summary of the day:** <The summary should be informative to the therapist. Keep the summary in 3-4 sentences/>
5. **Explanation:** <Explain your reasoning with detailed descriptions of the video and the person's state of mind. Break down your reasoning step-by-step. >
'''

In [11]:
def get_text_response(context, prompt):

    client = AzureOpenAI(
    azure_endpoint = OPENAI_ENDPOINT, 
    api_key=OPENAI_API_KEY,  
    api_version="2024-02-15-preview"
    )

    message_text = [
        {
            "role":"system",
            "content": context
        },
        {
            "role":"user",
            "content": prompt
        }
    ]

    completion = client.chat.completions.create(
        model="gpt-4-text", 
        messages = message_text,
        temperature=0.7,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )

    return completion

In [12]:
def get_image_response(context, image_path):

    encoded_image = base64.b64encode(open(image_path, 'rb').read()).decode('ascii')
    headers = {
        "Content-Type": "application/json",
        "api-key": OPENAI_API_KEY,
    }

    # Payload for the request
    payload = {
    "messages": [
        {
        "role": "system",
        "content": [
            {
            "type": "text",
            "text": context
            }
        ]
        },
        {
        "role": "user",
        "content": [
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}"
            }
            }
        ]
        }
    ],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 800
    }

    GPT4V_ENDPOINT = OPENAI_ENDPOINT+"/openai/deployments/gpt-4/chat/completions?api-version=2024-02-15-preview"

    # Send request
    try:
        response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    return response.json()['choices'][0]['message']['content']

In [13]:
def get_video_response(context, video_path):
    
    GPT_4V_ENDPOINT = os.environ("GPT_4V_ENDPOINT")
    GPT_4V_KEY = os.environ("GPT_4V_KEY")

    VISION_API_ENDPOINT = os.environ("VISION_API_ENDPOINT")
    VISION_API_KEY = os.environ("VISION_API_KEY")

    ## ingest the video
    VIDEO_FILE_SAS_URL = video_path
    VIDEO_INDEX_NAME = "Journal_analysis_video"
    VIDEO_DOCUMENT_ID = "AOAIChatDocument"

    def create_video_index(vision_api_endpoint, vision_api_key, index_name):
        url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}?api-version=2023-05-01-preview"
        headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
        data = {
            "features": [
                {"name": "vision", "domain": "surveillance"}
            ]
        }
        response = requests.put(url, headers=headers, json=data)
        return response

    def add_video_to_index(vision_api_endpoint, vision_api_key, index_name, video_url, video_id):
        url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions/my-ingestion?api-version=2023-05-01-preview"
        headers = {"Ocp-Apim-Subscription-Key": vision_api_key, "Content-Type": "application/json"}
        data = {
            'videos': [{'mode': 'add', 'documentId': video_id, 'documentUrl': video_url}]
        }
        response = requests.put(url, headers=headers, json=data)
        return response

    def wait_for_ingestion_completion(vision_api_endpoint, vision_api_key, index_name, max_retries=30):
        url = f"{vision_api_endpoint}/computervision/retrieval/indexes/{index_name}/ingestions?api-version=2023-05-01-preview"
        headers = {"Ocp-Apim-Subscription-Key": vision_api_key}
        retries = 0
        while retries < max_retries:
            time.sleep(10)
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                state_data = response.json()
                if state_data['value'][0]['state'] == 'Completed':
                    print(state_data)
                    print('Ingestion completed.')
                    return True
                elif state_data['value'][0]['state'] == 'Failed':
                    print(state_data)
                    print('Ingestion failed.')
                    return False
            retries += 1
        return False


    # Step 1: Create an Index
    response = create_video_index(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME)
    print(response.status_code, response.text)

    # Step 2: Add a video file to the index
    response = add_video_to_index(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME, VIDEO_FILE_SAS_URL, VIDEO_DOCUMENT_ID)
    print(response.status_code, response.text)

    # Step 3: Wait for ingestion to complete
    if not wait_for_ingestion_completion(VISION_API_ENDPOINT, VISION_API_KEY, VIDEO_INDEX_NAME):
        print("Ingestion did not complete within the expected time.")


    ## Chat with GPT-4V

    headers = {
        "Content-Type": "application/json",
        "api-key": GPT_4V_KEY,
    }

    # Payload for the request
    payload = {
        "dataSources": [
            {
                "type": "AzureComputerVisionVideoIndex",
                "parameters": {
                    "computerVisionBaseUrl": f"{VISION_API_ENDPOINT}/computervision",
                    "computerVisionApiKey": VISION_API_KEY,
                    "indexName": VIDEO_INDEX_NAME,
                    "videoUrls": [VIDEO_FILE_SAS_URL]
                }
            }
        ],
        "enhancements": {
            "video": {
                "enabled": True
            }
        },
        "messages": [
        {
            "role": "system",
            "content": [
                {
                        "type": "text",
                        "text": context
                }
            ]
        },
        {
            "role": "user",
            "content": [
                {
                        "type": "acv_document_id",
                        "acv_document_id": VIDEO_DOCUMENT_ID
                }
            ]
        }
    ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 800
    }

    # Send request
    try:
        response = requests.post(GPT_4V_ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    return response.json()['choices'][0]['message']['content']

 ## Journal View

This is what the journalling person views in the app.

In [14]:
# We are taking the inputs for the journal here.
# The person can use the input text boxes to enter his thoughts from the day

display(Markdown("## Daily Reflection Journal\n"))

# Printing out today's date

display(Markdown(date_display))

# We have given a few thought-provoking questions to let the person reflect on his day.
# The question can be skipped by just pressing the Enter key without writing anything.
# Make sure to answer at least one question.
# (We decided not to add the force input here. But if we develop an app over this we can
# make sure to include this feature).

experience = input("\nWhat emotions did I experience today and why?\n")
lessons = input("\nDid I learn any insights or lessons from the day?\n")
obstacles = input("\nWhat challenges or obstacles did I face today, and how did I approach them?\n")
reflections = input("\nWhat are the key reflections from the day?\n")
extra = input("\nDo you want to share anything else? Write a poem? Have a quote you relate to? Or just want to scribble some text?\n")

drawing = input("\nDid you draw anything today? If yes, share the file path of the drawing.\n")
video = input("\nWould you like to share a video? If yes, share the file path of the video.\n")

## Daily Reflection Journal


Tuesday - May 07, 2024

In [ ]:
# Writing all the answers into a file.
# This file will be accessible only for the person.
# The Therapist view will NOT have access to this file.

def writing_entry_to_file(entry, file_name):
  with open(os.path.join(data_folder, file_name), "a") as f:
    f.write(entry)

In [ ]:
journal_entry = f'''
----------------------------------------------------------
{date_display}

What emotions did I experience today and why?: {experience}
Did I learn any insights or lessons from the day?: {lessons}
What challenges or obstacles did I encounter today, and how did I approach them?: {obstacles}
What are the key reflections from the day?: {reflections}
Do you want to share anything else?: {extra}

Did you draw anything today? If yes, share only the file path of the drawing: data/sita-rama.jpg
'''

writing_entry_to_file(journal_entry, journal_file)

In [ ]:
print(journal_entry)


----------------------------------------------------------
Friday - May 03, 2024

What emotions did I experience today and why?: I was jubilant today! I saw a beautiful image of Sita and Rama walking in the woods, with love and compassion in the air. I felt so happy watching it. 
Did I learn any insights or lessons from the day?: No matter what we do, there's always positivity in the world. We need to search for it.
What challenges or obstacles did I encounter today, and how did I approach them?: 
What are the key reflections from the day?: Be happy in life!
Do you want to share anything else?: 

Did you draw anything today? If yes, share only the file path of the drawing: data/sita-rama.jpg



## Analysis

This is the part where GenAI comes into picture

In [52]:
def get_entry(file_name):
  if file_name not in os.listdir(data_folder):
    return "", ""

  with open(os.path.join(data_folder, file_name), "r") as f:
    text_in_file = f.read()

  text_entries = text_in_file.split('----------------------------------------------------------')

  if len(text_entries) < 2:
    return text_entries[-1], ""

  return text_entries[-1], text_entries[-2]

In [53]:
def get_journal_analysis(journal_today, journal_yesterday):

  # journal_today_parts = journal_today.split('\n\n')
  journal_yesterday_parts = journal_yesterday.split('\n\n')

  journal_yesterday = journal_yesterday_parts[1] if len(journal_yesterday_parts) > 1 else ''

  journal_prompt = journal_prompt_template.format(journal_yesterday=journal_yesterday, journal_today=journal_today)

  response = get_text_response(context=journal_context, prompt=journal_prompt)

  journal_analysis = f'''
----------------------------------------------------------
### {date_display}

## Analysis of the journal

{response}

'''

  return journal_analysis

In [54]:
def get_drawing_analysis(journal_today):

    journal_today_parts = journal_today.split('\n\n')
    image_name = journal_today_parts[-2].split(':')[-1].replace(' ', '').replace('\n', '')

    if image_name == '' or image_name is None:
        return ""

    response = get_image_response(context=drawing_context, image_path=image_name)

    drawing_analysis = f'''

    ## Analysis of the drawing

    {response.text}

    '''

    return drawing_analysis

In [55]:
def get_video_analysis(journal_today):

    journal_today_parts = journal_today.split('\n\n')
    video_name = journal_today_parts[-1].split(':')[-1].replace(' ', '').replace('\n', '')

    if video_name == '' or video_name is None:
        return ""

    response = get_video_response(context=drawing_context, video_path=video_name)

    video_analysis = f'''

    ## Analysis of the video

    {response.text}

    '''

    return video_analysis

In [56]:
journal_today, journal_yesterday = get_entry(journal_file)

In [60]:
get_journal_analysis()

In [ ]:
writing_entry_to_file(analysis_entry, analysis_file)

In [ ]:
display(Markdown(analysis_entry))

## Therapist View

This is what the therapist will see about the person.

In [ ]:
def get_analysis_entry(file_name, days_from_today = 0):
  if file_name not in os.listdir(data_folder):
    return [""]

  with open(os.path.join(data_folder, file_name), "r") as f:
    text_in_file = f.read()

  text_entries = text_in_file.split('----------------------------------------------------------')
  text_entries = text_entries[1:]

  if days_from_today < 0:
    return "Please enter a positive number (or 0 for today)"

  if len(text_entries) < days_from_today + 1:
    return "No Entry"

  return text_entries[-1-days_from_today]

In [ ]:
entry_day = input("Please enter the day of the analysis you want to see. 0 - today, 1 - yesterday, and so on...\n")
analysis_of_the_day = get_analysis_entry(analysis_file, days_from_today=int(entry_day))

Please enter the day of the analysis you want to see. 0 - today, 1 - yesterday, and so on...
0


In [ ]:
display(Markdown(analysis_of_the_day))


### Friday - May 03, 2024

### Analysis of the journal entry

## Analysis of Today's Journal Entry

**Emotions**: Joy, Hope, Love

**Possible thought patterns**:

* **Seeking Positivity**: The individual seems to actively search for positive aspects in the world around them, as evidenced by their reaction to the image of Sita and Rama. 
* **Focus on Love and Compassion**: The description of the image emphasizes love and compassion, suggesting these values are important to the individual and contribute to their positive emotional state.
* **Openness to Learning**:  The individual reflects on the day and derives a lesson about the importance of seeking positivity. This indicates a willingness to learn and grow from experiences.

**Mental well-being scores:**

| Mental State       | Confidence | Intensity |
| ----------------- | ---------- | -------- |
| Joy/Contentment    | 10         | 8        |
| Hopefulness        | 9          | 7        |
| Openness to learning| 8          | 6        |
| Spiritual well-being | 7          | 6        |
| Self-esteem       | 6          | 5        |
| Social connection  | 4          | 3        |
| Stress            | 3          | 2        |
| Anxiety           | 2          | 1        |
| Depression         | 1          | 1        |

**Summary of the day:**  Today's journal entry reflects a significant improvement in the individual's mental well-being compared to previous days.  The dominant emotions are joy, hope, and love, likely stemming from their focus on positive aspects of life and their appreciation for love and compassion. The individual demonstrates openness to learning and appears to be in a positive state of spiritual well-being.  

**Journal Excerpts:**

1. **"I was jubilant today!"**:  This excerpt clearly indicates a strong positive emotional state. 
2. **"I saw a beautiful image of Sita and Rama walking in the woods, with love and compassion in the air."**: This demonstrates the individual's focus on positivity and their appreciation for love and compassion.
3. **"No matter what we do, there's always positivity in the world. We need to search for it."**: This excerpt showcases the individual's belief in the presence of positivity and their active approach to seeking it out. 




  ### Analysis of the drawing

   1. **Emotions**: 
- Hope
- Love
- Contentment
- Optimism


2. **Possible thought patterns**: 
- The person is hopeful about the future.
- The person feels loved and supported.
- The person is content with their life.
- The person is optimistic about the future.


3. **Mental well-being scores:**

| Mental State | Confidence | Intensity |
| ------- | -------- | -------- |
| Hope | 10 | 8 |
| Love | 9 | 8 |
| Contentment | 8 | 7 |
| Optimism | 7 | 6 |
| Self-esteem | 6 | 5 |
| Self-worth | 5 | 4 |
| Self-confidence | 4 | 3 |
| Self-love | 3 | 2 |
| Happiness | 2 | 1 |
| Sadness | 1 | 0 |
| Anxiety | 0 | 0 |
| Stress | 0 | 0 |
| Depression | 0 | 0 |


4. **Summary of the day**: 
The person seems to be in a positive mental state. They are hopeful about the future, feel loved and supported, and are content with their life. However, the person may be experiencing low self-esteem and self-worth.


5. **Explanation**: 
The image depicts a person walking through a forest with a group of monkeys. The person is holding hands with a woman, who is smiling. The person is also smiling and looks happy. The monkeys are playing and jumping around, which suggests that the person is enjoying their company. The forest is green and lush, which suggests that the person is in a natural and peaceful environment. The overall tone of the image is positive and uplifting, which suggests that the person is in a good mental state.

  